### Backtesting.py 
### RSI no MA
#### https://kernc.github.io/backtesting.py/  

In [ ]:
!pip install backtesting
!pip install matplotlib

In [ ]:
#to import talib
#from backtesting.test import GOOG
from backtesting import Backtest, Strategy
import yfinance as yf
import pandas as pd
from backtesting.lib import crossover, plot_heatmaps, resample_apply, barssince
import talib
import seaborn as sns

In [ ]:
# Import symbol
symbols = ["KO"] 
df_multiple_symbols = yf.download(symbols, start="2020-01-01", end="2024-01-01")
# Flatten columns MultiIndex
if isinstance(df_multiple_symbols.columns, pd.MultiIndex):
    df_multiple_symbols.columns = df_multiple_symbols.columns.get_level_values(0)
df_multiple_symbols = df_multiple_symbols.dropna()


In [ ]:
df_multiple_symbols

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def optim_func(series):

#   filters out precondition 
    if series['Win Rate [%]'] < 75:
        return -1
    return series ['# Trades'] * series ['Return (Ann.) [%]'] 

class RsiOscillator(Strategy):
    
    upper_bound=80
    lower_bound=20
    rsi_window = 14
#    ema_window = 233
    
    #Rsi input from talib
    def init (self):
        self.rsi = self.I(talib.RSI, self.data.Close, self.rsi_window)
#        self.ema = self.I(talib.EMA, self.data.Close, self.ema_window)

    def next(self):
        if crossover(self.rsi, self.upper_bound):
            self.position.close()
        elif crossover(self.lower_bound, self.rsi): #and self.data.Close > self.ema:
            self.buy()

bt = Backtest(df_multiple_symbols, RsiOscillator, cash=10000)

#stats = bt.run()
stats, heatmap = bt.optimize(
    upper_bound = range(50, 90, 5),
    lower_bound = range(5, 50, 5),
    rsi_window = 2,
    # maximize towards
    maximize = "Sharpe Ratio",
    constraint = lambda param: param.upper_bound > param.lower_bound,
    return_heatmap = True
    # max_tries = 100
    )

#lower_bound = stats["_strategy"].lower_bound
print(heatmap)
hm = heatmap.groupby(["upper_bound", "lower_bound"]).mean().unstack()
#print(hm)
print(stats)
sns.heatmap(hm, cmap="plasma")
lower_bound = stats["_strategy"].lower_bound
upper_bound = stats["_strategy"].upper_bound
rsi_window = stats["_strategy"].rsi_window
bt.plot()

In [ ]:
# Process the heatmap to get top 50 results
if isinstance(heatmap, pd.Series):
    # Convert Series to DataFrame
    heatmap_df = heatmap.reset_index()
    print(f"Heatmap columns: {heatmap_df.columns.tolist()}")
    
    # Rename columns properly
    metric_col = heatmap_df.columns[-1]  
    heatmap_df = heatmap_df.rename(columns={
        heatmap_df.columns[0]: 'upper_bound',
        heatmap_df.columns[1]: 'lower_bound', 
        heatmap_df.columns[2]: 'rsi_window',
        metric_col: 'Sharpe Ratio'
    })
else:
    heatmap_df = heatmap.reset_index()

print(f"\nHeatmap shape: {heatmap_df.shape}")
print(f"Columns: {heatmap_df.columns.tolist()}")

# Get top 50 by Sharpe Ratio
top_50 = heatmap_df.nlargest(50, 'Sharpe Ratio').copy()

results_list = []

for idx, row in top_50.iterrows():

    temp_stats = bt.run(
        upper_bound=int(row['upper_bound']),
        lower_bound=int(row['lower_bound']),
        rsi_window=int(row['rsi_window'])
    )
    
    results_list.append({
        'rsi': int(row['rsi_window']),
        'bottom_value': int(row['lower_bound']),
        'upper_value': int(row['upper_bound']),
        'sharpe_ratio': float(row['Sharpe Ratio']),
        'winrate': float(temp_stats['Win Rate [%]']),
        'total_trades': int(temp_stats['# Trades'])
    })

results_df = pd.DataFrame(results_list)

print("\n" + "="*100)
print("TOP 50 RESULTS")
print("="*100)
print(f"{'RSI':^6} {'Bottom':^8} {'Upper':^8} {'Sharpe Ratio':^12} {'Win Rate %':^10} {'Trades':^8}")
print("-"*60)

for _, row in results_df.iterrows():
    print(f"{row['rsi']:^6} {row['bottom_value']:^8} {row['upper_value']:^8} {row['sharpe_ratio']:^12.3f} {row['winrate']:^10.2f} {row['total_trades']:^8}")

print("="*100)
print(f"Total results shown: {len(results_df)}")
print("="*100)

# Also print as a table for easier copying
print("\nResults as DataFrame (for analysis):")
print(results_df.to_string(index=False))